# AIAP11 EDA

### Loading required libraries

In [302]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from src.extract import ImportData
from config.config_load import read_yaml_file

### Loading Dataset

In [303]:
# Reading in data location from config file
config = read_yaml_file()
data_location = config["data"]["data_location"]

# Calling the data from module 1
table = ImportData(data_location).return_table("noshow")
table

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,1.0,Changi,July,May,19.0,May,20.0,China,Yes,King,None,Email,2,1.0
1,0.0,Orchard,December,February,28.0,March,-4.0,India,Yes,None,USD$ 723.34,Email,1,0.0
2,0.0,Orchard,December,May,22.0,May,24.0,Australia,Yes,Single,SGD$ 650.94,Website,1,0.0
3,0.0,Orchard,October,September,2.0,September,4.0,China,Yes,None,SGD$ 978.67,Agent,1,1.0
4,0.0,Orchard,February,February,7.0,February,8.0,China,Yes,King,None,Website,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119386,0.0,Changi,October,March,17.0,March,19.0,China,Yes,King,SGD$ 953.94,Email,2,1.0
119387,0.0,Changi,August,June,2.0,June,3.0,Indonesia,Yes,King,USD$ 636.84,Email,1,1.0
119388,0.0,Changi,January,July,1.0,July,3.0,Indonesia,Yes,King,None,Website,1,0.0
119389,0.0,Changi,January,May,26.0,May,29.0,Indonesia,Yes,King,USD$ 720.1,Email,1,1.0


### Basic Data Description

Wanted to look at:
1. Types of categories within the dataset and if they were classified correctly 
2. Column names and information if it tallies with information provided by AISG 
3. Any other salient information that might suggest how data was captured and how it was put together

In [304]:
num_cat = table.select_dtypes(include=[np.number])
cat_cat = table.select_dtypes(include="object")
for each in cat_cat:
    table[each] = table[each].astype("category")

print(f"There are {table.shape[1]} columns in the dataset with {table.shape[0]} observations")
print(f"There are {num_cat.shape[1]} numerical columns and {cat_cat.shape[1]} categorical columns in the dataset")
print(f"There are {table.isna().sum().sum()} empty observations in dataset")

There are 14 columns in the dataset with 119391 observations
There are 4 numerical columns and 10 categorical columns in the dataset
There are 46507 empty observations in dataset


In [305]:
table.columns

Index(['no_show', 'branch', 'booking_month', 'arrival_month', 'arrival_day',
       'checkout_month', 'checkout_day', 'country', 'first_time', 'room',
       'price', 'platform', 'num_adults', 'num_children'],
      dtype='object')

In [306]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119391 entries, 0 to 119390
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   no_show         119390 non-null  float64 
 1   branch          119390 non-null  category
 2   booking_month   119390 non-null  category
 3   arrival_month   119390 non-null  category
 4   arrival_day     119390 non-null  float64 
 5   checkout_month  119390 non-null  category
 6   checkout_day    119390 non-null  float64 
 7   country         119390 non-null  category
 8   first_time      119390 non-null  category
 9   room            97778 non-null   category
 10  price           94509 non-null   category
 11  platform        119390 non-null  category
 12  num_adults      119390 non-null  category
 13  num_children    119390 non-null  float64 
dtypes: category(10), float64(4)
memory usage: 7.6 MB


In [307]:
table.first_time.unique()
# print(table.isna().sum())

['Yes', 'No', NaN]
Categories (2, object): ['No', 'Yes']

### Data Cleaning and Preprocessing
Observations
1. Price is recorded with currency and value, this was seperated to allow for better comparison
2. There was one row that was empty within the dataset row number: 115536
3. "num_adults" was a freetext field hence [1,2,"one", "two"] within series
4. Several columns was saved as fload in database but should have been integers ["arrival_day", "checkout_day", "num_children"]
5. "first_time" should be converted to boolean [1,0]
6. 

In [308]:
# Identify the prices which have "None" and replace with np.nan
table.price.replace(["None", "nan"], np.nan)

# Split out currency from room price
table["currency"] = table.price.apply(lambda x: np.nan if x=="None" else x[:3]).astype("category")

# Split out price from price
def split_price(price):
    """Parse out the relevant price from the series
    Args:
        price (string): price with full currency and price 
    Returns:
        price: float point price
    """
    if price == "None" or price is None:
        return np.nan
    else:
        return float(price[5:])

table["price"] = table.price.apply(lambda x: split_price(x))

In [309]:
# table shape before
print(f"table shape before processing {table.shape}")

# Viewing sliced table with empty data
table.loc[table.no_show.isna()]

# Removing empty data line
# table.iloc[table.index[115536], :]
table.drop(table.index[115536], inplace=True)

# Checking removal
print(f"table shape after processing {table.shape}")

table shape before processing (119391, 15)
table shape after processing (119390, 15)


In [310]:
# changing number of adults to integers
def chg_to_num(num):
    """Change numbers from string to integers for use in lambda only
    Args:
        num (string): string version of integer
    Returns:
        num: numerical version of string integer
    """
    if num is None:
        return np.nan
    elif num=="one":
        return int(1)
    elif num=="two":
        return int(2)
    else:
        return int(num)

table["num_adults"] = table.num_adults.apply(lambda x: chg_to_num(x))

In [311]:
# Changed all other data from float to categorical
table["num_adults"] = table["num_adults"].astype("int")
table["arrival_day"] = table["arrival_day"].astype("int")
table["checkout_day"] = table["checkout_day"].astype("int")
table["num_children"] = table["num_children"].astype("int")

# change first_time to either 1 for "Yes" or 0 for "No"
table["first_time"] = table["first_time"].apply(lambda x: int(1) if x =="Yes" else int(0))


table.first_time.unique()


[1, 0]
Categories (2, int64): [0, 1]

In [312]:
table

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children,currency
0,1.0,Changi,July,May,19,May,20,China,1,King,NaN,Email,2,1,NaN
1,0.0,Orchard,December,February,28,March,-4,India,1,NaN,723.34,Email,1,0,USD
2,0.0,Orchard,December,May,22,May,24,Australia,1,Single,650.94,Website,1,0,SGD
3,0.0,Orchard,October,September,2,September,4,China,1,NaN,978.67,Agent,1,1,SGD
4,0.0,Orchard,February,February,7,February,8,China,1,King,NaN,Website,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119386,0.0,Changi,October,March,17,March,19,China,1,King,953.94,Email,2,1,SGD
119387,0.0,Changi,August,June,2,June,3,Indonesia,1,King,636.84,Email,1,1,USD
119388,0.0,Changi,January,July,1,July,3,Indonesia,1,King,NaN,Website,1,0,NaN
119389,0.0,Changi,January,May,26,May,29,Indonesia,1,King,720.10,Email,1,1,USD


### Exploratory Data Analysis